In [1]:
import json
import logging
import os
import pandas as pd
import random
import re
import sys
from datetime import date
from dotenv import load_dotenv
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils.utils import GPT

In [2]:
# params
# st_mdl = "gpt-3.5-turbo-1106" # model string
st_mdl = "gpt-4-1106-preview"
st_task = "piqa" # name of task

# logging
## today's date
st_today = date.today().strftime("%Y_%m_%d")

## file name:
st_fn = f"{st_today}_{st_task}_n200_{st_mdl}"

## Configure the logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## Create a file handler and set the formatter
file_handler = logging.FileHandler(f"results/{st_fn}.log")
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

## Create a logger instance and add the file handler
logger = logging.getLogger(__name__)
logger.addHandler(file_handler)

In [3]:
# load api key from .env and create GPT object
load_dotenv()  # Load environment variables from .env file
openai_api_key = os.getenv("OPENAI_API_KEY")
gpt = GPT(openai_api_key, logger=logger, model=st_mdl)

# Load into a dictionary
data = []
with open('data/valid.jsonl', 'r') as files:
    for f in files:
        data.append(json.loads(f))

with open('data/valid-labels.lst', 'r') as f:
    ans = [a.strip() for a in f.readlines()]


# dataset-specific adjustments (we eventually want a list of dicts of format [{"id": "xx", "question": "xxx", "context": "xxx"}, ...]

# pre-process data

data = [{'question': d['goal'], 'options': [d['sol1'], d['sol2']], 'answer': a} for d, a in zip(data, ans)]
    
# data = pd.read_csv('data/24game.csv').to_dict(orient='records') # load CSV into list of dicts

In [4]:
def compare_strings(str1, str2):
    """
    This function compares two strings after converting them to lowercase and stripping leading and trailing spaces.
    """
    return str1.lower().strip() == str(str2).lower().strip()

def check_answer(pred, gold):
    """
    This function compares first int in str pred with int gold
    """
    try:
        # return int(re.search(r'\d+', pred).group()) == int(gold)
        return pred[0] == map_int_to_char(gold)
    except:
        return False

def map_int_to_char(num):
    """
        Convert int num to string (e.g. 0 maps to A, 1 maps to B
    """
    return chr(num + ord('A'))

def check_bool(str1, bool2):
    """
        Checks if string str1 equal to bool bool2
    """
    try:
        return bool(str1) == bool2
    except:
        return False
        

In [5]:
# visualize dataset

df = pd.json_normalize(data)
df.head()

,question,options,answer
0,How do I ready a guinea pig cage for it's new ...,[Provide the guinea pig with a cage full of a ...,0
1,dresser,"[replace drawer with bobby pin , finish, woodg...",1
2,To fight Ivan Drago in Rocky for sega master s...,[Drago isn't in this game because it was relea...,1
3,Make outdoor pillow.,"[Blow into tin can and tie with rubber band., ...",1
4,ice box,[will turn into a cooler if you add water to i...,0


## Generator

In [10]:
predictions = [] # list of predictions, reset after each set of runs
indices = [] # question numbers
answers = [] # list of gold answers
evals = [] # whether prediction matches answer
questions = [] # list of questions


# Evaluate
num_questions = 200 # number of questions to run on
indices = random.sample(range(len(data)), num_questions)
for idx in indices:
    item = data[idx]
    # if item['level'] == 'hard':
    # gold, question, context = item['answer'], item['question'], item['context']
    gold, q, opts = item['answer'], item['question'], item['options']

    # concat question with options
    question = f"{q}\nA.{opts[0]}\nB.{opts[1]}\n"
    logger.info(question)
    
    msg = [{"role": "system", "content": "Please answer the following multiple choice question. Respond only with one of {A, B} corresponding to the correct answer```"}, {"role": "user", "content": question}]
    pred = gpt.gen(msgs=msg).content
    predictions.append(pred)
    answers.append(gold)
    # evals.append(compare_strings(predictions[-1], gold))
    evals.append(check_answer(predictions[-1], gold))
    questions.append(question)
    print(predictions[-1])

# Print the predictions and answers
logger.info(predictions)
# logger.info(answers)
percent_correct = (sum(evals) / len(evals)) * 100

print(f"Percentage of True values: {percent_correct}%")

# create a dataframe with the data
df = pd.DataFrame({'Index': indices, 'Question': questions, 'Prediction': predictions, 'Answer': answers, 'Eval': evals})
# df = pd.DataFrame({'Index': indices, 'Question': questions, 'Prediction': predictions, 'Eval': evals})

# save the datafrme to csv
df.to_csv(f'results/{st_fn}.csv', index=False)

2024-02-14 12:17:26,881 - INFO - Deter mosquitoes from camp fire.
A.Burn sage in the fire.
B.Burn twigs in the fire.

2024-02-14 12:17:27,515 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 12:17:27,517 - INFO - Attach tooth picks to a cork board to create a thread spool holder.
A.Place tooth picks pointed side down into cork board at 3 inch intervals. Lightly tap the top of the tooth pick at a horizontal angle with a rubber mallet to anchor into the cork board to create a thread spool holder.
B.Place tooth picks pointed side down into cork board at 3 inch intervals. Lightly tap the top of the tooth pick at a horizontal angle with a rubber mallet to anchor into the cookie sheet to create a thread spool holder.



A.Burn sage in the fire.


2024-02-14 12:17:28,248 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 12:17:28,250 - INFO - how do you open a wrapped present?
A.tear off all the wrapping.
B.cut off the wrapping paper.



A


2024-02-14 12:17:28,959 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 12:17:28,961 - INFO - What can you do when someone presses all the buttons on the elevator then gets off and you are in a hurry?
A.You can avoid stopping at all the floors by pressing the ones you want to skip twice, it will avoid the stop there.
B.You can avoid passing all the floors by pressing the ones you want to skip twice, it will avoid the passing of the elevator there.



...

B


2024-02-14 12:19:50,331 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 12:19:50,333 - INFO - How to make Vanilla Confetti Fantasy fudge at home.
A.Bring mixture to a full boil for 5 minutes, stirring constantly.  Remove from heat and stir in 12 ounce white chocolate chips until melted and thoroughly combined. Beat in 1 7 ounce jar of marshmallow creme, and 1/4 cup paper confetti .  Transfer fudge to Greased 8 x 13" pan and let cool before adding more sprinkles to the top and cutting into squares.
B.Bring mixture to a full boil for 5 minutes, stirring constantly.  Remove from heat and stir in 12 ounce white chocolate chips until melted and thoroughly combined. Beat in 1 7 ounce jar of marshmallow creme, and 1/4 cup confetti Sprinkles.    Transfer fudge to Greased 8 x 13" pan and let cool before adding more sprinkles to the top and cutting into squares.



B


2024-02-14 12:19:51,026 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 12:19:51,027 - INFO - ['A.Burn sage in the fire.', 'A', 'A', 'B. You can avoid passing all the floors by pressing the ones you want to skip twice, it will avoid the passing of the elevator there.', 'A', 'A', 'B', 'A', 'B', 'B', 'B', 'A', 'A', 'A', 'B', 'B.Wrap bright cloth around luggage handle.', 'A', 'B', 'B', 'B', 'A. Use a thick guitar pick as a pry along the edges of the case to open it.', 'A.can color a paper towel roll green', 'B', 'B', 'A.', 'B', 'B', 'A', 'B', 'A.', 'A', 'Your question does not provide enough context for me to determine a definitive answer. Are you referring to the purpose of a specific gun type, a law, a regulation, or some other context? In most situations, guns can be used for a variety of purposes, including hunting different types of animals. Without further context, I cannot provide an accurate response to your multiple-choice quest

B
Percentage of True values: 0.0%


In [24]:
df = pd.read_csv(f"results/{st_fn}.csv")
# df['Answer'] = df['Answer'].apply(lambda x: x.split('\n#### ')[-1])
df['Eval'] = df.apply(lambda row: check_answer(row['Prediction'], row['Answer']), axis=1)

df.to_csv(f'results/{st_fn}.csv', index=False)
df.head()

,Index,Question,Prediction,Answer,Eval
0,1765,Deter mosquitoes from camp fire.\nA.Burn sage ...,A.Burn sage in the fire.,0,True
1,810,Attach tooth picks to a cork board to create a...,A,0,True
2,680,how do you open a wrapped present?\nA.tear off...,A,0,True
3,68,What can you do when someone presses all the b...,B. You can avoid passing all the floors by pre...,0,False
4,759,How do I prepare my parchment paper when makin...,A,0,True


## Validator

### Baseline

In [ ]:
fn_sfx = "" # filename suffix, used when saving validator file
sys_prompt = "Answer 'TRUE' if the answer is correct and 'FALSE' if the answer is incorrect."

### Search for optimal prompt

In [6]:
fn_sfx = "_prmpt" # filename suffix, used when saving validator file
# sys_prompt = "Think step by step. First generate your own answer to the question. Next reflect on your answer. Finally, compare this with the provided answer. Check, then double check your thinking. The last word of your response should be 'TRUE' if the answer is correct and 'FALSE' if the answer is incorrect, given the question."
sys_prompt = "Think step by step. First generate your own answer to the question and then compare this with the provided answer. Check, then double check your thinking. The last word of your response should be 'TRUE' if the answer is correct and 'FALSE' if the answer is incorrect, given the question."

### Run validator

In [7]:
gen_data_fn = f"results/{st_fn}.csv" # change this to the filename of the generator results
# gen_data_fn = 'results/2024_02_13_piqa_n200_gpt-3.5-turbo-1106.csv'

df = pd.read_csv(gen_data_fn)
indices = df.iloc[:, 0].tolist()
indices = indices[:-1] # just because we manually added an additional row on the CSV when calculating scores

val_preds = [] # list of predictions for validator, reset after each set of runs
val_evals = [] # whether prediction matches answer


# Evaluate
for _, row in df.iterrows():
    idx = row['Index']
    ans = row['Prediction']
    gold = row['Eval'] # change to Manual Eval in other settings
    question = row['Question']
    
    q = "Question: " + question + " "
    q += f"Answer: {ans}"
    logger.info(q)
    msg = [{"role": "system", "content": "The following are a series of multiple choice questions. Respond 'TRUE' if the answer is correct and 'FALSE' if incorrect. Respond only with one word."}, {"role": "user", "content": q}]
    val_preds.append(gpt.gen(msgs=msg).content)
    val_evals.append(check_bool(val_preds[-1], gold))
    print(val_preds[-1])

# Print the predictions and answers
logger.info(val_preds)
logger.info(df['Eval'].tolist())
percent_correct = (sum(val_evals) / len(val_evals)) * 100

print(f"Percentage of True values: {percent_correct}%")

# add validator predictions to column
df['Validator Pred'] = val_preds

# save the dataframe to csv
df.to_csv(f'results/{st_fn}_with_val{fn_sfx}.csv', index=False);

2024-02-14 18:10:10,232 - INFO - Question: Deter mosquitoes from camp fire.
A.Burn sage in the fire.
B.Burn twigs in the fire.
 Answer: A.Burn sage in the fire.
2024-02-14 18:10:11,088 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 18:10:11,090 - INFO - Question: Attach tooth picks to a cork board to create a thread spool holder.
A.Place tooth picks pointed side down into cork board at 3 inch intervals. Lightly tap the top of the tooth pick at a horizontal angle with a rubber mallet to anchor into the cork board to create a thread spool holder.
B.Place tooth picks pointed side down into cork board at 3 inch intervals. Lightly tap the top of the tooth pick at a horizontal angle with a rubber mallet to anchor into the cookie sheet to create a thread spool holder.
 Answer: A


TRUE


2024-02-14 18:10:11,457 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 18:10:11,460 - INFO - Question: how do you open a wrapped present?
A.tear off all the wrapping.
B.cut off the wrapping paper.
 Answer: A


TRUE


2024-02-14 18:10:11,772 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 18:10:11,774 - INFO - Question: What can you do when someone presses all the buttons on the elevator then gets off and you are in a hurry?
A.You can avoid stopping at all the floors by pressing the ones you want to skip twice, it will avoid the stop there.
B.You can avoid passing all the floors by pressing the ones you want to skip twice, it will avoid the passing of the elevator there.
 Answer: B. You can avoid passing all the floors by pressing the ones you want to skip twice, it will avoid the passing of the elevator there.


...

FALSE


2024-02-14 18:12:13,812 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 18:12:13,814 - INFO - Question: How to make Vanilla Confetti Fantasy fudge at home.
A.Bring mixture to a full boil for 5 minutes, stirring constantly.  Remove from heat and stir in 12 ounce white chocolate chips until melted and thoroughly combined. Beat in 1 7 ounce jar of marshmallow creme, and 1/4 cup paper confetti .  Transfer fudge to Greased 8 x 13" pan and let cool before adding more sprinkles to the top and cutting into squares.
B.Bring mixture to a full boil for 5 minutes, stirring constantly.  Remove from heat and stir in 12 ounce white chocolate chips until melted and thoroughly combined. Beat in 1 7 ounce jar of marshmallow creme, and 1/4 cup confetti Sprinkles.    Transfer fudge to Greased 8 x 13" pan and let cool before adding more sprinkles to the top and cutting into squares.
 Answer: B


TRUE


2024-02-14 18:12:14,297 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 18:12:14,299 - INFO - ['TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', '

TRUE
Percentage of True values: 90.5%
